### Installing Required Packages

In [34]:
!pip install requests

### Importing the packages


In [13]:
import os
from pathlib import Path # for handling path issues
import logging # for creating the execution log in directory
import requests # for sending http requests
import sqlite3 as _dbsql # for handling SQ-Lite3 database implementation
from http.server import BaseHTTPRequestHandler, HTTPServer # for implementing a Simple HTTP Server in Python

### Execution Constants and Variables
* For handling database and log implementation, the path constraints are handled in the below components
* The data base file is checked if it exists, and the dB file is created.
* The logger is configured to store the logs in a specific format


In [14]:
_db_root = "Database"

In [15]:
_dir = Path().resolve()

In [16]:
_log = logging.getLogger(__name__)
_log.setLevel(logging.DEBUG)
iohandler = logging.StreamHandler()
iohandler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
iohandler.setFormatter(formatter)
_log.addHandler(iohandler)
logging.basicConfig(filename="execution_log.log", filemode='w')

### Changing path to current directory (Execute only once)
* Here, the dB file is checked for schema existence and if there is no data-base schema then a table is created with Device name as primary key and Data, timestamp as column values.

In [17]:
try:
    os.chdir(_dir)
    _db_root_check = os.path.isdir(_db_root)
    if not _db_root_check:
        os.makedirs(_db_root)
    _db_path = os.path.join(_dir, _db_root)
    os.chdir(_db_path)
    _log.info("database root initialised successfully")
except Exception as _initialization_exception:
    _log.critical("root path could not be initialised")

2022-02-18 11:04:08,657 - __main__ - INFO - database root initialised successfully
2022-02-18 11:04:08,657 - __main__ - INFO - database root initialised successfully


# Database Initialization

In [18]:
try:
    _db_con = _dbsql.connect('inm_warehouse.db')
    _log.info("database inm_warehouse.db initialised successfully")
except Exception as _db_initialization:
    _log.critical("database could not be initialised")

2022-02-18 11:04:09,046 - __main__ - INFO - database inm_warehouse.db initialised successfully
2022-02-18 11:04:09,046 - __main__ - INFO - database inm_warehouse.db initialised successfully


In [19]:
try:
    _db_cursor = _db_con.cursor()
    _db_cursor.execute('''CREATE TABLE warehouse
               (device_name text not null, timestamp text not null, data text not null)''')
    _log.info("database table creation successful")
except Exception as _db_exception:
    _log.warning("database table could not be initialised: " + str(_db_exception))

2022-02-18 11:04:09,511 - __main__ - INFO - database table creation successful
2022-02-18 11:04:09,511 - __main__ - INFO - database table creation successful


#### Now two functions are built, one for retrieving the descriptor information from the OM2M Server and one for retrieving the data from Sq-Lite3 dB file.

### Get Descriptor for a node name from OM2M

In [20]:
url = "http://127.0.0.1:5089/~/in-cse/in-name/AE-TEST/Node-1/Descriptor/la"
payload={}
headers = {'X-M2M-Origin': 'admin:admin','Accept': 'application/json'}
response = requests.request("GET", url, headers=headers, data=payload)
data=eval(response.text)
data

{'m2m:cin': {'rn': 'cin_404229808',
  'ty': 4,
  'ri': '/in-cse/cin-404229808',
  'pi': '/in-cse/cnt-434525523',
  'ct': '20220218T080407',
  'lt': '20220218T080407',
  'lbl': ['Label-1', 'Label-2'],
  'st': 0,
  'cnf': 'text/plain:0',
  'cs': 38,
  'con': '[Timestamp, Occupancy-State, Distance]'}}

In [21]:
def get_descriptor(Node_name):
        url = "http://127.0.0.1:5089/~/in-cse/in-name/AE-TEST/" + Node_name + "/Descriptor/la"
        payload={}
        headers = {'X-M2M-Origin': 'admin:admin','Accept': 'application/json'}
        response = requests.request("GET", url, headers=headers, data=payload)
        if response.status_code != 200:
            _log.warning("device not found")
            return False, None
        data = eval(response.text)
        con = data["m2m:cin"]["con"].replace(" ", "")
        con = con.split("[")[1]
        con = con.split("]")[0]
        con = con.split(",")
        _log.info("data retrieved from om2m")
        return True, con

### Get data from database for a given node-name

In [22]:
def get_from_db(_node_name):
    name = "'" + _node_name + "'"
    _data = _db_cursor.execute('SELECT * FROM warehouse where device_name =' + name)
    _log.info("data retrieved from db")
    return _data

In [25]:
name = "'" + 'Node-1' + "'"
_data = _db_cursor.execute('SELECT * FROM warehouse where device_name =' + name)
_data

In [26]:
for i in _data:
    print(i)

('Node-1', '1645162500', '[1645162500, 1.0, 338.0]')
('Node-1', '1645162505', '[1645162505, 1.0, 262.0]')
('Node-1', '1645162511', '[1645162511, 1.0, 348.0]')
('Node-1', '1645162516', '[1645162516, 0.0, 303.0]')
('Node-1', '1645162521', '[1645162521, 1.0, 271.0]')
('Node-1', '1645162526', '[1645162526, 1.0, 48.0]')
('Node-1', '1645162531', '[1645162531, 0.0, 225.0]')
('Node-1', '1645162537', '[1645162537, 0.0, 107.0]')
('Node-1', '1645162542', '[1645162542, 1.0, 224.0]')
('Node-1', '1645162547', '[1645162547, 1.0, 74.0]')
('Node-1', '1645162552', '[1645162552, 0.0, 398.0]')
('Node-1', '1645162557', '[1645162557, 0.0, 43.0]')
('Node-1', '1645162562', '[1645162562, 1.0, 315.0]')
('Node-1', '1645162567', '[1645162567, 0.0, 125.0]')
('Node-1', '1645162572', '[1645162572, 1.0, 100.0]')
('Node-1', '1645162578', '[1645162578, 0.0, 79.0]')
('Node-1', '1645162583', '[1645162583, 1.0, 285.0]')
('Node-1', '1645162588', '[1645162588, 1.0, 310.0]')
('Node-1', '1645162593', '[1645162593, 0.0, 222.0]

### Server Class for data warehousing
* Now a Simple HTTP Server is implemented with a post and get request.
* Post request is used for storing the subscription data in the Sq-Lite3 DB file by implementing a data warehouse system
* GET request is used for retrieving a data relevant to  a node name from the data warehouse through an API call.


In [23]:
class handler(BaseHTTPRequestHandler):
    
    
    def do_GET(self):
        
        if "/ngsi-ld/v1/entities/" in self.path:
            _node_name = self.path.split("/ngsi-ld/v1/entities/")[1]
            _parameter_list = ["node_id"]
            _om2m_status = get_descriptor(_node_name)
            
            if not _om2m_status[0]:
                self.send_response(404)
                response = "{\"type\":\"urn:dx:rs:general\",\"title\":\"Device Not Registered\",\"detail\":\"Device Not Registered\"}"
            
            else:
                _lis = []
                self.send_response(200)
                _parameter_list += _om2m_status[1]
                _log.info(_parameter_list)
                response = "{\"title\": \"Successful operation\", \"type\": \"urn:dx:rs:success\", \"results\": ["
                
                _data = get_from_db(_node_name)
                for row in _data:
                    _log.info(row)
                    _lis += [row]
                _ordered_data = list(reversed(_lis))
                length_of_data = len(_ordered_data)
                for row_id in range(length_of_data):
                    _current_row = [_node_name]
                    _current_row += eval(list(_ordered_data[row_id])[2])
                    response += "{"
                    _data_length = len(_parameter_list)
                    
                    for parameter_id in range(_data_length):
                        response += "\"" + _parameter_list[parameter_id] + "\":" + "\"" + str(_current_row[parameter_id])
                        
                        if parameter_id == _data_length -1:
                            response += "\""
                        
                        else:
                            response += "\","
                    
                    if row_id == length_of_data -1:
                        response += "}"
                    
                    else:
                        response += "},"
                
                response += "]}"
                _log.info("data formatting successsful")
                    
        else:
            _log.warning("url-end point not found")
            self.send_response(404)
            response = "{\"type\":\"urn:dx:rs:general\",\"title\":\"NotFound\",\"detail\":\"NotFound\"}"
            
        self.send_header('Content-type','application/json')
        self.end_headers()
        self.wfile.write(bytes(response, "utf8"))
    
    def do_POST(self):
        self.data_string = self.rfile.read(int(self.headers['Content-Length'])).decode("utf-8")
        self.send_response(200)
        self.send_header('Content-type','text/html')
        self.end_headers()
        try:
            var = eval(self.data_string.replace("false", "False" ))
            _con = eval(var["m2m:sgn"]["m2m:nev"]["m2m:rep"]["m2m:cin"]["con"])
            _lbl = var["m2m:sgn"]["m2m:nev"]["m2m:rep"]["m2m:cin"]["lbl"]
            _values = (str(_lbl[0]), str(_con[0]), str(_con))
            _query = "INSERT INTO warehouse VALUES " + str(_values)
            _db_cursor.execute(_query)
            _log.info("data stored with query: " + str(_query))
        except Exception as ioerror:
            _log.critical("data storage failed with exception: " + str(ioerror))
        
        message = "{\"Status\": 200}"
        self.wfile.write(bytes(message, "utf8"))

In [24]:
with HTTPServer(('', 5091), handler) as server:
        server.serve_forever()

2022-02-18 11:04:37,350 - __main__ - INFO - data retrieved from om2m
2022-02-18 11:04:37,350 - __main__ - INFO - data retrieved from om2m
127.0.0.1 - - [18/Feb/2022 11:04:37] "GET /ngsi-ld/v1/entities/Node-1 HTTP/1.1" 200 -
2022-02-18 11:04:37,354 - __main__ - INFO - ['node_id', 'Timestamp', 'Occupancy-State', 'Distance']
2022-02-18 11:04:37,354 - __main__ - INFO - ['node_id', 'Timestamp', 'Occupancy-State', 'Distance']
2022-02-18 11:04:37,359 - __main__ - INFO - data retrieved from db
2022-02-18 11:04:37,359 - __main__ - INFO - data retrieved from db
2022-02-18 11:04:37,367 - __main__ - INFO - data formatting successsful
2022-02-18 11:04:37,367 - __main__ - INFO - data formatting successsful
2022-02-18 11:05:00,828 - __main__ - INFO - data retrieved from om2m
2022-02-18 11:05:00,828 - __main__ - INFO - data retrieved from om2m
127.0.0.1 - - [18/Feb/2022 11:05:00] "GET /ngsi-ld/v1/entities/Node-1 HTTP/1.1" 200 -
2022-02-18 11:05:00,837 - __main__ - INFO - ['node_id', 'Timestamp', 'Occu

KeyboardInterrupt: 